In [3]:
%matplotlib inline
import pandas as pd

In [4]:
import numpy as np

In [8]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [7]:
homes = pd.DataFrame.from_csv('Data_Files/Zip_Zhvi_AllHomes.csv', index_col=None)
homes.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05
0,61639,10025,New York,NY,New York,New York,1,NaN,NaN,NaN,...,1128500,1133300,1132900,1147300,1149800.0,1132700,1121400,1123000,1143700,1178700
1,84654,60657,Chicago,IL,Chicago,Cook,2,146700.0,146500.0,146300.0,...,318600,318200,318100,318700,320200.0,321000,322200,323800,325900,327700
2,84616,60614,Chicago,IL,Chicago,Cook,3,198000.0,195500.0,194200.0,...,400700,401900,406000,414000,417800.0,417700,418700,414100,405400,401900
3,93144,79936,El Paso,TX,El Paso,El Paso,4,70800.0,71000.0,71000.0,...,113700,113800,113800,114100,114500.0,114900,115000,114700,114700,114800
4,84640,60640,Chicago,IL,Chicago,Cook,5,102300.0,101300.0,100700.0,...,200000,198800,199200,200100,201500.0,203200,205200,206700,206600,205700


In [44]:
homes2 = pd.melt(homes, id_vars=["RegionID", "RegionName", "City", "State", "Metro", "CountyName", "SizeRank"], var_name="Date", value_name="Value")
homes2.tail(20).sort_values('Value')

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,Date,Value
3328658,85883,63041,Grubville,MO,St. Louis,Franklin,13094,2017-05,61700.0
3328650,78648,47528,Edwardsport,IN,Vincennes,Knox,13086,2017-05,67100.0
3328653,78867,47965,Crawfordsville,IN,Crawfordsville,Montgomery,13089,2017-05,81600.0
3328656,67087,21865,Tyaskin,MD,Salisbury,Wicomico,13092,2017-05,105700.0
3328659,67072,21840,Nanticoke,MD,Salisbury,Wicomico,13095,2017-05,112200.0
3328652,67086,21864,Stockton,MD,Salisbury,Worcester,13088,2017-05,122400.0
3328654,59187,3765,Haverhill,NH,Claremont,Grafton,13090,2017-05,124100.0
3328651,81916,54832,Drummond,WI,NaN,Bayfield,13087,2017-05,153800.0
3328667,59107,3293,Woodstock,NH,Claremont,Grafton,13103,2017-05,168900.0
3328662,62556,12429,Port Ewen,NY,Kingston,Ulster,13098,2017-05,175600.0


In [26]:
len(homes2)

3328670

In [27]:
len(homes)

13105

In [ ]:
# plot annual average growth for all US in the last 20 years

In [41]:
h = homes2
h = h.groupby(h.Date)
h['average'] = h.apply(average, axis=1)
h.Value.plot()


NameError: name 'average' is not defined

In [ ]:
# show cumulative growth by state for the last 20 years

In [58]:
h = homes2
g = h.groupby(['State', 'Date']).average()
g

AttributeError: 'DataFrameGroupBy' object has no attribute 'average'

In [ ]:
# show annual growth by state for the last 20 years

In [ ]:
# plot annual growth by state for 5 slowest growing states for the last 20 years

In [ ]:
# plot annual growth by state for 5 slowest growing states for the last 20 years

In [ ]:
# plot annual growth by zip for 5 fastest growing zips for the last 20 years

In [ ]:
# plot annual growth by zip for 5 slowest growing zips for the last 20 years